In [80]:

import json
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import google_key
gkey = google_key
gmaps.configure(gkey)



In [81]:
City_Weather = "output_data/cities.csv"
City_Weather_d = pd.read_csv(City_Weather)
#open csv
#read_csv

In [82]:
#mapea con weight
City_Weather_d

,City Id,City,Cloudiness,Country,Date,Humidity,lattitude,long,Max temp,Wind Speed
0,1,bambous virieux,29,MU,1632245731,70,-20.3428,57.7575,70.21,13.94
1,2,nelson bay,81,AU,1632245732,91,-32.7167,152.1500,50.95,22.57
2,3,shingu,70,JP,1632245732,94,33.7333,135.9833,79.92,3.78
3,4,elat,0,IL,1632245733,22,29.5581,34.9482,93.04,17.27
4,5,miles city,1,US,1632245733,25,46.4083,-105.8406,66.78,11.68
...,...,...,...,...,...,...,...,...,...,...
541,542,kampot,59,KH,1632245940,88,10.6167,104.1833,77.56,5.12
542,543,maceio,40,BR,1632245941,66,-9.6658,-35.7353,87.24,11.50
543,544,manono,40,CD,1632245941,22,-7.3000,27.4167,86.67,4.21
544,545,yarada,99,IN,1632245942,80,17.6500,83.2667,82.96,4.83


In [83]:

locations = City_Weather_d[["lattitude", "long"]]
Humid = City_Weather_d["Humidity"].astype(float)
locations

,lattitude,long
0,-20.3428,57.7575
1,-32.7167,152.1500
2,33.7333,135.9833
3,29.5581,34.9482
4,46.4083,-105.8406
...,...,...
541,10.6167,104.1833
542,-9.6658,-35.7353
543,-7.3000,27.4167
544,17.6500,83.2667


In [84]:
maxv = Humid.max()

In [85]:
maxv

100.0

In [86]:
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights= Humid,
                                 dissipating=False, max_intensity=maxv,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [87]:
City_Weather_GoodTemp = City_Weather_d.loc[(City_Weather_d["Max temp"]< 80) & (City_Weather_d["Max temp"] >70) &
                                           (City_Weather_d["Wind Speed"] <10) & (City_Weather_d["Cloudiness"] == 0)]



In [88]:
City_Weather_GoodTemp

,City Id,City,Cloudiness,Country,Date,Humidity,lattitude,long,Max temp,Wind Speed
13,14,aksu,0,CN,1632245736,21,41.1231,80.2644,73.36,3.87
16,17,windhoek,0,NaN,1632245738,14,-22.5594,17.0832,76.10,5.99
70,71,ambilobe,0,MG,1632245758,70,-13.2000,49.0500,71.71,5.32
137,138,naze,0,JP,1632245783,76,28.3667,129.4833,75.79,3.85
221,222,saint george,0,US,1632245791,17,37.1041,-113.5841,77.79,1.99
293,294,dutlwe,0,BW,1632245842,8,-23.9833,23.9000,70.66,7.96
363,364,salt,0,JO,1632245869,46,32.0392,35.7272,73.27,4.43
397,398,huaihua,0,CN,1632245883,74,27.5494,109.9592,71.17,1.90
400,401,lata,0,PT,1632245885,33,40.1629,-8.3327,77.36,8.57
421,422,kaka,0,TM,1632245893,23,37.3482,59.6143,74.07,5.50


In [89]:
City_Weather_GoodTemp = City_Weather_GoodTemp.dropna()

In [90]:
City_W = pd.DataFrame({
    'City': City_Weather_GoodTemp['City'],
    'Country': City_Weather_GoodTemp['Country'],
    'lattitude': City_Weather_GoodTemp['lattitude'],
    'long'   : City_Weather_GoodTemp['long']})

In [91]:
City_W["Hotel"]=""

In [92]:
City_W

,City,Country,lattitude,long,Hotel
13,aksu,CN,41.1231,80.2644,
70,ambilobe,MG,-13.2000,49.0500,
137,naze,JP,28.3667,129.4833,
221,saint george,US,37.1041,-113.5841,
293,dutlwe,BW,-23.9833,23.9000,
363,salt,JO,32.0392,35.7272,
397,huaihua,CN,27.5494,109.9592,
400,lata,PT,40.1629,-8.3327,
421,kaka,TM,37.3482,59.6143,
423,khash,IR,28.2211,61.2158,


In [93]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {

    "radius": 5000,
    "type": "lodging",
    "keyword": "Hotel",
    "key": gkey}


for index, row in City_W.iterrows():
   
    lat = row["lattitude"]
    lng = row["long"]
    params["location"] = f"{lat},{lng}"
        
    response = requests.get(base_url, params=params).json()
     
    results = response['results']
    try:
        print(f"Closest Hotel to {row['City']}City in {row['Country']} is {results[0]['name']}.")
        City_W.loc[index, "Hotel"] = results[0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Closest Hotel to aksuCity in CN is Pudong Holiday Hotel.
Closest Hotel to ambilobeCity in MG is Hotel National.
Closest Hotel to nazeCity in JP is 山羊島ホテル.
Closest Hotel to saint georgeCity in US is St. George Inn & Suites.
Missing field/result... skipping.
Closest Hotel to saltCity in JO is Saltus Hotel.
Closest Hotel to huaihuaCity in CN is Kairuida Hotel.
Missing field/result... skipping.
Missing field/result... skipping.
Closest Hotel to khashCity in IR is مسافر.
Closest Hotel to nanchangCity in CN is Sheraton Nanchang Hotel.


In [94]:
City_W

,City,Country,lattitude,long,Hotel
13,aksu,CN,41.1231,80.2644,Pudong Holiday Hotel
70,ambilobe,MG,-13.2000,49.0500,Hotel National
137,naze,JP,28.3667,129.4833,山羊島ホテル
221,saint george,US,37.1041,-113.5841,St. George Inn & Suites
293,dutlwe,BW,-23.9833,23.9000,
363,salt,JO,32.0392,35.7272,Saltus Hotel
397,huaihua,CN,27.5494,109.9592,Kairuida Hotel
400,lata,PT,40.1629,-8.3327,
421,kaka,TM,37.3482,59.6143,
423,khash,IR,28.2211,61.2158,مسافر


In [95]:
len(City_W)

11

In [96]:
City_W = City_W.dropna()

In [97]:
len(City_W)

11

In [98]:
Hotel_name = City_W["Hotel"].tolist()

In [99]:
info_box_template = """

 Name: {Hotel}
 City: {City}
 Country: {Country}
  
"""
hotel_info = [info_box_template.format(**row) for index, row in City_W.iterrows()]

# Create a map using state centroid coordinates to set markers
marker_locations = City_W[['lattitude', 'long']]


fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
                             hotel_info)
#    info_box_content=[f"Hotel name: {Hotel}" for Hotel in Hotel_name])


fig.add_layer(heat_layer)
fig.add_layer(markers)
fig




Figure(layout=FigureLayout(height='420px'))

In [100]:

#The majority hotels are close in latitude, during the summer months in the northen hemiphire it places them far 
#enough from the equator so the temperature. 
#Is not too hot, yet close enough that the temperature is not too cold ether 
#This would change at different time of the year, like during winter months these hotels would probably 
#bee too cold to meet the same criteria.
